In [23]:
import csv
import json

import random
import itertools
import math
import numpy as np
import pandas as pd

import psycopg2

import gmaps
import gmaps.geojson_geometries
from geographiclib.geodesic import Geodesic

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

In [5]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [6]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from customers
order by customer_id

"""

customer_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
customer_df

,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance
0,1,Robb,Weaving,5 Ramsey Place,Oakland,CA,94609,1,1
1,2,Robby,Belliard,6 Londonderry Plaza,Oakland,CA,94609,1,1
2,3,Sadella,Caudrelier,548 Mcguire Parkway,Oakland,CA,94609,1,1
3,4,Holmes,Shimmings,99 Kennedy Court,Oakland,CA,94609,1,1
4,5,Beverley,Gubbin,51 Mcbride Drive,Oakland,CA,94609,1,1
...,...,...,...,...,...,...,...,...,...
31077,31078,Hugo,Domeney,529 5th Plaza,Thompsons Station,TN,37179,5,25
31078,31079,Glenn,Putson,1347 Westend Crossing,Thompsons Station,TN,37179,5,25
31079,31080,Minnie,Antham,9 Judy Place,Thompsons Station,TN,37179,5,25
31080,31081,Linet,Djorvic,29 Trailsway Drive,Thompsons Station,TN,37179,5,25


In [7]:
bay_customers_df = customer_df[(customer_df["city"] == "Oakland") | (customer_df["city"] == "San Francisco") | (customer_df["city"] == "Berkeley")]
bay_customers_df

,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance
0,1,Robb,Weaving,5 Ramsey Place,Oakland,CA,94609,1,1
1,2,Robby,Belliard,6 Londonderry Plaza,Oakland,CA,94609,1,1
2,3,Sadella,Caudrelier,548 Mcguire Parkway,Oakland,CA,94609,1,1
3,4,Holmes,Shimmings,99 Kennedy Court,Oakland,CA,94609,1,1
4,5,Beverley,Gubbin,51 Mcbride Drive,Oakland,CA,94609,1,1
...,...,...,...,...,...,...,...,...,...
7636,7637,Samara,Chester,1 Dottie Circle,San Francisco,CA,94132,1,15
7637,7638,Frasier,Ryle,39165 John Wall Plaza,San Francisco,CA,94132,1,15
7638,7639,Wilbert,Hew,9397 Stone Corner Trail,San Francisco,CA,94132,1,15
7639,7640,Rianon,Cowlas,7 Merrick Alley,San Francisco,CA,94132,1,15


In [8]:
bay_customers_df["city"].value_counts()

Oakland          1928
San Francisco    1437
Berkeley         1077
Name: city, dtype: int64

In [9]:
bay_customers_df["zip"].value_counts()

94602    260
94611    204
94607    203
94606    199
94601    180
94610    176
94702    171
94612    161
94704    161
94618    156
94609    155
94703    155
94705    134
94707    119
94709    117
94708    115
94102     78
94123     77
94103     77
94110     75
94109     75
94710     75
94107     74
94124     70
94115     68
94134     64
94131     63
94114     58
94603     58
94621     58
94133     57
94158     56
94619     55
94605     54
94118     52
94117     52
94108     50
94105     48
94122     41
94129     40
94112     40
94116     40
94132     37
94111     36
94121     36
94127     36
94130     31
94720     30
94613      9
94104      5
94128      1
Name: zip, dtype: int64

Pick 3 zip codes: 94602, 94133, 94132

Get the geographic center for each of the chosen zip codes.

In [10]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from zip_codes
where zip = '94602' or zip = '94133' or zip = '94132'

"""

zip_code_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
zip_code_df

,zip,latitude,longitude,city,state,population,area,density,time_zone
0,94132,37.7222,-122.4849,San Francisco,CA,31488,3.4813,9045.02,America/Los_Angeles
1,94133,37.8038,-122.4107,San Francisco,CA,26527,0.7634,34748.33,America/Los_Angeles
2,94602,37.8041,-122.2070,Oakland,CA,29933,3.4924,8570.79,America/Los_Angeles


In [11]:
"""
Geographic Centers (latitude, longitude) for Zip Codes that will be Used for Random Location Generation
"""
point_94602 = (37.8041, -122.2070)

point_94133 = (37.8038, -122.4107)

point_94132 = (37.7222, -122.4849)

In [12]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [13]:
"""Calculate a 1-mile box around 94602"""

box_94602 = my_calculate_box(point_94602, 1)
box_94602

((37.80409858265136, -122.22527433288882),
 (37.80409858265136, -122.18872566711117),
 (37.81859948472099, -122.207),
 (37.78960047949969, -122.207))

In [14]:
"""Calculate a 1-mile box around 94133"""

box_94133 = my_calculate_box(point_94133, 1)
box_94133

((37.803798582666595, -122.42897425896916),
 (37.803798582666595, -122.39242574103085),
 (37.818299485461345, -122.4107),
 (37.789300478759465, -122.4107))

In [15]:
"""Calculate a 1-mile box around 94132"""

box_94132 = my_calculate_box(point_94132, 1)
box_94132

((37.72219858680375, -122.50315419362695),
 (37.72219858680375, -122.46664580637304),
 (37.73669968675821, -122.4849),
 (37.70770027748825, -122.4849))

In [16]:
"""Generate Location from 94602"""
left_94602, right_94602, top_94602, bottom_94602 = box_94602
loc_94602 = ( random.uniform( left_94602[0], right_94602[0] ), random.uniform( bottom_94602[1], top_94602[1] ) )
loc_94602

(37.80409858265136, -122.207)

In [17]:
"""Generate Location from 94133"""
left_94133, right_94133, top_94133, bottom_94133 = box_94133
loc_94133 = ( random.uniform( left_94133[0], right_94133[0] ), random.uniform( bottom_94133[1], top_94133[1] ) )
loc_94133

(37.803798582666595, -122.4107)

In [18]:
"""Generate Location from 94132"""
left_94132, right_94132, top_94132, bottom_94132 = box_94132
loc_94132 = ( random.uniform( left_94132[0], right_94132[0] ), random.uniform( bottom_94132[1], top_94132[1] ) )
loc_94132

(37.72219858680375, -122.4849)

In [19]:
#
#  Given two points in (latitude, longitude) format, calculate the distance between them in miles
#

def my_calculate_distance(point_1, point_2):
    "Given two points in (latitude, longitude) format, calculate the distance between them in miles"
    
    geod = Geodesic.WGS84


    g = geod.Inverse(point_1[0], point_1[1], point_2[0], point_2[1])
    miles = g['s12'] / 1000 * 0.621371
    
    return miles

In [20]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from stores

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,store_id,street,city,state,zip,latitude,longitude
0,1,3000 Telegraph Ave,Berkeley,CA,94705,37.8555,-122.2604
1,2,1001 Broadway,Seattle,WA,98122,47.6114,-122.3214
2,3,2510 McKinney Ave,Dallas,TX,75201,32.7958,-96.8015
3,4,299 SE 3rd Ave,Miami,FL,33131,25.7720,-80.1891
4,5,1202 Broadway,Nashville,TN,37203,36.1568,-86.7881


In [22]:
"""Our Starting Position"""
loc_berkeley_store = (37.8555, -122.2604)

94602 - Piedmont
94133 - Lombard St.
94132 - Lakeshore

In [25]:
start_end_point = "Berkeley AGM"

#delivery points
delivery_locations = ["Piedmont", "Lombard St.", "Lakeshore"]

#find all permuations of delivery points
permutations_delivery_locations = list(itertools.permutations(delivery_locations))

permutations_delivery_locations

[('Piedmont', 'Lombard St.', 'Lakeshore'),
 ('Piedmont', 'Lakeshore', 'Lombard St.'),
 ('Lombard St.', 'Piedmont', 'Lakeshore'),
 ('Lombard St.', 'Lakeshore', 'Piedmont'),
 ('Lakeshore', 'Piedmont', 'Lombard St.'),
 ('Lakeshore', 'Lombard St.', 'Piedmont')]

In [26]:
# add start to beginning and end of every permutation

all_possible_paths = []

for d in permutations_delivery_locations:
    all_possible_paths.append([start_end_point] + list(d) + [start_end_point])
    
all_possible_paths

[['Berkeley AGM', 'Piedmont', 'Lombard St.', 'Lakeshore', 'Berkeley AGM'],
 ['Berkeley AGM', 'Piedmont', 'Lakeshore', 'Lombard St.', 'Berkeley AGM'],
 ['Berkeley AGM', 'Lombard St.', 'Piedmont', 'Lakeshore', 'Berkeley AGM'],
 ['Berkeley AGM', 'Lombard St.', 'Lakeshore', 'Piedmont', 'Berkeley AGM'],
 ['Berkeley AGM', 'Lakeshore', 'Piedmont', 'Lombard St.', 'Berkeley AGM'],
 ['Berkeley AGM', 'Lakeshore', 'Lombard St.', 'Piedmont', 'Berkeley AGM']]

In [27]:
travel_times = {}
    
travel_times['Berkeley AGM to Piedmont'] = my_calculate_distance(loc_berkeley_store, loc_94602)
travel_times['Berkeley AGM to Lombard St.'] = my_calculate_distance(loc_berkeley_store, loc_94133)
travel_times['Berkeley AGM to Lakeshore'] = my_calculate_distance(loc_berkeley_store, loc_94132)

travel_times['Piedmont to Lombard St.'] = my_calculate_distance(loc_94602, loc_94133)
travel_times['Piedmont to Lakeshore'] = my_calculate_distance(loc_94602, loc_94132)
travel_times['Piedmont to Berkeley AGM'] = my_calculate_distance(loc_94602, loc_berkeley_store)

travel_times['Lombard St. to Piedmont'] = my_calculate_distance(loc_94133, loc_94602)
travel_times['Lombard St. to Lakeshore'] = my_calculate_distance(loc_94133, loc_94132)
travel_times['Lombard St. to Berkeley AGM'] = my_calculate_distance(loc_94133, loc_berkeley_store)

travel_times['Lakeshore to Piedmont'] = my_calculate_distance(loc_94132, loc_94602)
travel_times['Lakeshore to Lombard St.'] = my_calculate_distance(loc_94132, loc_94133)
travel_times['Lakeshore to Berkeley AGM'] = my_calculate_distance(loc_94132, loc_berkeley_store)

In [28]:
"""Find the Travel Times"""
def my_get_travel_time(start, end):
    return(travel_times[start + " to " + end])

In [32]:
# find the path with the lowest total cost

cost_all_possible_paths = []

for p in all_possible_paths:
    total_cost = 0
    for i in range(0, len(p)-1):        
        cost = my_get_travel_time(p[i], p[i+1])
        total_cost += cost
    cost_all_possible_paths.append([total_cost,p])

cost_all_possible_paths.sort()

cost_all_possible_paths

[[36.726161716285716,
  ['Berkeley AGM', 'Lombard St.', 'Lakeshore', 'Piedmont', 'Berkeley AGM']],
 [36.726161716285716,
  ['Berkeley AGM', 'Piedmont', 'Lakeshore', 'Lombard St.', 'Berkeley AGM']],
 [38.027240319172094,
  ['Berkeley AGM', 'Lakeshore', 'Lombard St.', 'Piedmont', 'Berkeley AGM']],
 [38.027240319172094,
  ['Berkeley AGM', 'Piedmont', 'Lombard St.', 'Lakeshore', 'Berkeley AGM']],
 [51.68463540052369,
  ['Berkeley AGM', 'Lakeshore', 'Piedmont', 'Lombard St.', 'Berkeley AGM']],
 [51.684635400523696,
  ['Berkeley AGM', 'Lombard St.', 'Piedmont', 'Lakeshore', 'Berkeley AGM']]]

The delivery run with the shortest travel distance will be from Berkeley AGM to Lombard St., Lombard St. to Lakeshore, Lakeshore to Piedmont, and Piedmont back to Berkeley AGM.